<a href="https://colab.research.google.com/github/ltdaovn/analog-meter-reading-openCV/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%matplotlib inline

!pip install keras
!pip install tensorflow  
!pip3 install opencv-python
!pip install pypng
!pip install matplotlib


from google.colab.patches import cv2_imshow
from skimage import data,filters
from PIL import ImageTk, Image


import cv2
import csv
import matplotlib.pyplot as plt
import matplotlib.pylab as plt
import matplotlib.cm as cm
import numpy as np
import pathlib
import PIL
import os,png,array
import pandas as pd
import time

In [ ]:
!wget https://github.com/ltdaovn/analog-meter-reading-openCV/raw/main/cnn_svhn_.model

--2021-10-17 07:59:36--  https://github.com/ltdaovn/analog-meter-reading-openCV/raw/main/cnn_svhn_.model
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ltdaovn/analog-meter-reading-openCV/main/cnn_svhn_.model [following]
--2021-10-17 07:59:37--  https://raw.githubusercontent.com/ltdaovn/analog-meter-reading-openCV/main/cnn_svhn_.model
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3773544 (3.6M) [application/octet-stream]
Saving to: ‘cnn_svhn_.model’

cnn_svhn_.model     100%[===================>]   3.60M  --.-KB/s    in 0.07s   

2021-10-17 07:59:37 (53.2 MB/s) - ‘cnn_svhn_.model’ saved [3773

In [ ]:
!wget "https://github.com/ltdaovn/analog-meter-reading-openCV/raw/main/images.zip"

--2021-10-17 07:59:37--  https://github.com/ltdaovn/analog-meter-reading-openCV/raw/main/images.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ltdaovn/analog-meter-reading-openCV/main/images.zip [following]
--2021-10-17 07:59:37--  https://raw.githubusercontent.com/ltdaovn/analog-meter-reading-openCV/main/images.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9240431 (8.8M) [application/zip]
Saving to: ‘images.zip’

images.zip          100%[===================>]   8.81M  --.-KB/s    in 0.1s    

2021-10-17 07:59:38 (80.6 MB/s) - ‘images.zip’ saved [9240431/9240431]



In [ ]:
!unzip images.zip

Archive:  images.zip
   creating: images/
  inflating: images/SAYAC (1).jpg    
  inflating: images/SAYAC (10).jpg   
  inflating: images/SAYAC (100).jpg  
  inflating: images/SAYAC (101).jpg  
  inflating: images/SAYAC (102).jpg  
  inflating: images/SAYAC (103).jpg  
  inflating: images/SAYAC (104).jpg  
  inflating: images/SAYAC (105).jpg  
  inflating: images/SAYAC (106).jpg  
  inflating: images/SAYAC (107).jpg  
  inflating: images/SAYAC (108).jpg  
  inflating: images/SAYAC (109).jpg  
  inflating: images/SAYAC (11).jpg   
  inflating: images/SAYAC (110).jpg  
  inflating: images/SAYAC (111).jpg  
  inflating: images/SAYAC (112).jpg  
  inflating: images/SAYAC (113).jpg  
  inflating: images/SAYAC (114).jpg  
  inflating: images/SAYAC (115).jpg  
  inflating: images/SAYAC (116).jpg  
  inflating: images/SAYAC (117).jpg  
  inflating: images/SAYAC (118).jpg  
  inflating: images/SAYAC (119).jpg  
  inflating: images/SAYAC (12).jpg   
  inflating: images/SAYAC (120).jpg  
  inflat

In [ ]:
from keras.models import load_model
classifier=load_model('cnn_svhn_.model')

row_list = []
index_counter = 0
resize_counter = 0

directory="images/"

for file in os.listdir(directory):
  path = directory + file
  print(path)
  try:

    if file.endswith(".jpg"): 
        img = cv2.imread(path, 0)

        if img is not None:
            #blur = cv2.GaussianBlur(grey, value, 0)
            img = cv2.GaussianBlur(img, (3,3), 0)

        edges = cv2.Canny(img, 175, 200, apertureSize=3, L2gradient=False)
        def auto_canny(image, sigma=0.33):

            v = np.median(image)
            lower = int(max(0, (1.0 - sigma) * v))
            upper = int(min(255, (1.0 + sigma) * v))
            edged = cv2.Canny(image, lower, upper, apertureSize=3, L2gradient=True)
            return edged

        #edges = auto_canny(img)

        contours, hierarchy = cv2.findContours(edges.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        contours_dict = dict()
        for cont in contours:
            x, y, w, h = cv2.boundingRect(cont)
            area = cv2.contourArea(cont)
            if 20 < area and 20 < w and h > 10 :
                contours_dict[(x, y, w, h)] = cont

        contours_filtered = sorted(contours_dict.values(), key=cv2.boundingRect)
        blank_background = np.zeros_like(edges)
        img_contours = cv2.drawContours(blank_background, contours_filtered, -1, (255,255,255), thickness=2)

        def is_overlapping_horizontally(box1, box2):
            x1, _, w1, _ = box1
            x2, _, _, _ = box2
            if x1 > x2:
                return is_overlapping_horizontally(box2, box1)
            return (x2 - x1) < w1

        def merge(box1, box2):
            assert is_overlapping_horizontally(box1, box2)
            x1, y1, w1, h1 = box1
            x2, y2, w2, h2 = box2
            x = min(x1, x2)
            w = max(x1 + w1, x2 + w2) - x
            y = min(y1, y2)
            h = max(y1 + h1, y2 + h2) - y
            return (x, y, w, h)

        def windows(contours):
            boxes = []
            for cont in contours:
                box = cv2.boundingRect(cont)
                if not boxes:
                    boxes.append(box)
                else:
                    if is_overlapping_horizontally(boxes[-1], box):
                        last_box = boxes.pop()
                        merged_box = merge(box, last_box)
                        boxes.append(merged_box)
                    else:
                        boxes.append(box)
            return boxes

        boxes = windows(contours_filtered)
        len_boxes = len(boxes)

        for n in range(len_boxes):
          x, y, w, h = boxes[n]
          plt.axis('off')
          
          if y < 10:
              y = 10
          if x < 10:
              x = 10
              
          roi = img[y-10:y+h+5, x-10:x+w+5]
          roi = cv2.resize(roi,(32, 32),interpolation = cv2.INTER_AREA)
          a = str(n+1)
          cv2.imwrite('basamak'+a+'.png', roi)

        columnNames = list()

        for i in range(1024):
            pixel = 'pixel'
            pixel += str(i)
            columnNames.append(pixel)

        train_data = pd.DataFrame(columns = columnNames)

        for n in range(len_boxes):
            a = str(n+1)
            b='basamak'+a+'.png'
            img = Image.open(b)
            rawData = img.load()
            data = []
            for y in range(32):
                for x in range(32):
                    data.append(rawData[x,y])
            k = 0
            train_data.loc[0] = [data[k] for k in range(1024)]
            train_data = train_data.div(255)
            train_data.to_csv("train_converted"+a+".csv",index = False)


        w ={}
        for n in range(len_boxes):
            a = str(n+1)
            b = "train_converted"+a+".csv"
            x_pred = pd.read_csv(b)
            x_pred = x_pred.iloc[:,:].values.astype('float32')
            x_pred =x_pred.reshape(-1, 32, 32, 1)
            predictions=classifier.predict(x_pred)
            x = predictions[4]
            c = "predictions"+a
            w.update( {c : x.argmax()} )
        
        row = [file,  str(w.get("predictions1")) + 
               str(w.get("predictions2")) +  
               str(w.get("predictions3")) +
               str(w.get("predictions4")) +
               str(w.get("predictions5")) +
               str(w.get("predictions6")) ]

        row_list.append(row)

        print( file + " " + "SAYAÇ MİKTAR: " +
            str(w.get("predictions1")) + 
            str(w.get("predictions2")) +  
            str(w.get("predictions3")) +
            str(w.get("predictions4")) +
            str(w.get("predictions5")) +
            str(w.get("predictions6")) 
            )

        continue
        
    else:
        continue

  except IndexError:
    if IndexError:
      index_counter += 1
      print(file + " " +'List index out of range error')
      print('index count: ', index_counter)
      pass

  '''except:
    row = [file, "Resize Error"]
    resize_counter += 1
    print('resize count: ', resize_counter)
    print(file + " " + 'resize error')'''

In [ ]:
print(row_list)

[['SAYAC (105).jpg', '0030NoneNone'], ['SAYAC (90).jpg', '00690None'], ['SAYAC (114).jpg', '002360'], ['SAYAC (23).jpg', '20046None'], ['SAYAC (49).jpg', '000620'], ['SAYAC (92).jpg', '203698'], ['SAYAC (95).jpg', '007300'], ['SAYAC (149).jpg', '00850None'], ['SAYAC (48).jpg', '000650'], ['SAYAC (148).jpg', '200828'], ['SAYAC (34).jpg', '60560None'], ['SAYAC (167).jpg', '000533'], ['SAYAC (76).jpg', '003660'], ['SAYAC (96).jpg', '200736'], ['SAYAC (165).jpg', '000579'], ['SAYAC (104).jpg', '00240None'], ['SAYAC (63).jpg', '003630'], ['SAYAC (109).jpg', '200350'], ['SAYAC (5).jpg', '600522'], ['SAYAC (84).jpg', '003685'], ['SAYAC (112).jpg', '00010None'], ['SAYAC (54).jpg', '00660None'], ['SAYAC (32).jpg', '205620'], ['SAYAC (89).jpg', '006910'], ['SAYAC (14).jpg', '000530'], ['SAYAC (85).jpg', '206850'], ['SAYAC (61).jpg', '006260'], ['SAYAC (29).jpg', '000558'], ['SAYAC (39).jpg', '20520None'], ['SAYAC (68).jpg', '000633'], ['SAYAC (135).jpg', '002880'], ['SAYAC (52).jpg', '60620None'

In [ ]:
# Create a file for new predicted values, then write it
# update this csv each time you changed something in the code above 
# then follow these steps and calculate the accuracy, compare with other results
with open('analog_predicted_meter.csv', 'w', newline='') as file:
  writer = csv.writer(file)
  writer.writerows(row_list)
print(len(row_list))

169


In [ ]:
print("-------------------------------------")
print("resize counter error:", " ", resize_counter)
print("index counter error:" , " " , index_counter)
print("-------------------------------------")
read = 169 - (int(resize_counter) + int(index_counter))
not_read = int(resize_counter) + int(index_counter)
print("reading: ", read)
print("not reading: ", not_read)
print("-------------------------------------")

-------------------------------------
resize counter error:   0
index counter error:   0
-------------------------------------
reading:  169
not reading:  0
-------------------------------------


In [ ]:
read_value = pd.read_csv("analog_predicted_meter.csv")
read_value.columns = ["imageID", "read_value"]
read_value["imageID"] = read_value["imageID"].str.replace(r'.jpg$', '')
#read_val["ImageID"] = read_val["ImageID"].str.replace(r'.png$', '')

# Delete basamak.png rows
#read_val = read_val[read_val.ReadValue == basamak]

read_value

,imageID,read_value
0,SAYAC (90),00690None
1,SAYAC (114),002360
2,SAYAC (23),20046None
3,SAYAC (49),000620
4,SAYAC (92),203698
...,...,...
163,SAYAC (55),006170
164,SAYAC (115),00230None
165,SAYAC (70),606420
166,SAYAC (79),00630None


In [ ]:
real_value = pd.read_excel("analog_actual_meter.xlsx", dtype={'actual_value':np.str})
real_value

In [ ]:
merged = pd.merge(read_value, real_value, on="imageID")
%load_ext google.colab.data_table 
merged

In [ ]:
merged["read_value"] = merged["read_value"].str.replace(r'None', 'N')
merged

In [ ]:
merged['IfMatch'] = np.where(merged['read_value'] == merged['actual_value'], 'True', 'False')
merged[merged['IfMatch'] == 'True']
#merged

In [ ]:
merged["Total"] = merged['read_value'] 

for row in range(167):

  match_counter = 0

  for basamak in range(6):
    if merged['read_value'][row][basamak]== merged['actual_value'][row][basamak]:
      match_counter += 1

  merged['Total'][row] = match_counter

In [ ]:
merged["digit_correctness_of_each_meter"] = (merged["Total"]/6)*100
 
print("accuracy:", (sum (merged["Total"])/ (6*167))*100)

merged